In [3]:
#hide
from fastai2.vision.all import *
from fastai2.vision.widgets import *

import ipywidgets as widgets
from IPython.display import display,clear_output

In [13]:
RED = '\033[31m'
BLUE = '\033[94m'
GREEN = '\033[92m'
BOLD   = '\033[1m'
ITALIC = '\033[3m'
RESET  = '\033[0m'

style = {'description_width': 'initial'}

In [9]:
def dashboard_one():
    """GUI for first accordion window"""
    import psutil
    import torchvision
    try:
        import fastai2; fastver = fastai2.__version__
    except ImportError:
        fastver = 'fastai not found'
    try:
        import fastprogress; fastprog = fastprogress.__version__
    except ImportError:
        fastprog = 'fastprogress not found'
    try:
        import fastpages; fastp = fastpages.__version__
    except ImportError:
        fastp = 'fastpages not found'
    try:
        import nbdev; nbd = nbdev.__version__
    except ImportError:
        ndb = 'nbdev not found'

    print (BOLD +  RED + '>> Vision_UI Update: 03/17/2020')
    style = {'description_width': 'initial'}

    button = widgets.Button(description='System', button_style='success')
    ex_button = widgets.Button(description='Explore', button_style='success')
    display(button)

    out = widgets.Output()
    display(out)

    def on_button_clicked_info(b):
        with out:
            clear_output()
            print(BOLD + BLUE + "fastai2 Version: " + RESET + ITALIC + str(fastver))
            print(BOLD + BLUE + "nbdev Version: " + RESET + ITALIC + str(nbd))
            print(BOLD + BLUE + "fastprogress Version: " + RESET + ITALIC + str(fastprog))
            print(BOLD + BLUE + "fastpages Version: " + RESET + ITALIC + str(fastp) + '\n')
            print(BOLD + BLUE + "python Version: " + RESET + ITALIC + str(sys.version))
            print(BOLD + BLUE + "torchvision: " + RESET + ITALIC + str(torchvision.__version__))
            print(BOLD + BLUE + "torch version: " + RESET + ITALIC + str(torch.__version__))
            print(BOLD + BLUE + "\nCuda: " + RESET + ITALIC + str(torch.cuda.is_available()))
            print(BOLD + BLUE + "cuda Version: " + RESET + ITALIC + str(torch.version.cuda))
            print(BOLD + BLUE + "GPU: " + RESET + ITALIC + str(torch.cuda.get_device_name(0)))
            print(BOLD + BLUE + "\nCPU%: " + RESET + ITALIC + str(psutil.cpu_percent()))
            print(BOLD + BLUE + "\nmemory % used: " + RESET + ITALIC + str(psutil.virtual_memory()[2]))
    button.on_click(on_button_clicked_info)


In [14]:
def dashboard_two():
    """GUI for second accordion window"""
    dashboard_two.datas = widgets.ToggleButtons(
        options=['PETS', 'CIFAR', 'IMAGENETTE_160', 'IMAGEWOOF_160', 'MNIST_TINY'],
        description='Choose',
        value=None,
        disabled=False,
        button_style='info',
        tooltips=[''],
        style=style
    )

    display(dashboard_two.datas)

    button = widgets.Button(description='Explore', button_style='success')
    display(button)
    out = widgets.Output()
    display(out)
    def on_button_explore(b):
        with out:
            clear_output()
            ds_choice()

    button.on_click(on_button_explore)

In [16]:
def ds_choice():
    """Helper for dataset choices"""
    if dashboard_two.datas.value == 'PETS':
        ds_choice.source = untar_data(URLs.DOGS)
    elif dashboard_two.datas.value == 'CIFAR':
        ds_choice.source = untar_data(URLs.CIFAR)
    elif dashboard_two.datas.value == 'IMAGENETTE_160':
        ds_choice.source = untar_data(URLs.IMAGENETTE_160)
    elif dashboard_two.datas.value == 'IMAGEWOOF_160':
        ds_choice.source = untar_data(URLs.IMAGEWOOF_160)
    elif dashboard_two.datas.value == 'MNIST_TINY':
        ds_choice.source = untar_data(URLs.MNIST_TINY)

    print(BOLD + BLUE + "Dataset: " + RESET + BOLD + RED + str(dashboard_two.datas.value))
    plt_classes()

In [83]:
def plt_classes():
    """Helper for plotting classes in folder"""
    disp_img_but = widgets.Button(description='View Images?', button_style='success')
    Path.BASE_PATH = ds_choice.source
    train_source = (ds_choice.source/'train/').ls().items
    print(BOLD + BLUE + "Folders: " + RESET + BOLD + RED + str(train_source))
    print(BOLD + BLUE + "\n" + "No of classes: " + RESET + BOLD + RED + str(len(train_source)))

    num_l = []
    class_l = []
    for j, name in enumerate(train_source):
        fol = (ds_choice.source/name).ls().sorted()
        names = str(name)
        class_split = names.split('train')
        class_l.append(class_split[1])
        num_l.append(len(fol))

    y_pos = np.arange(len(train_source))
    performance = num_l
    fig = plt.figure(figsize=(10,5))

    plt.style.use('seaborn')
    plt.bar(y_pos, performance, align='center', alpha=0.5, color=['black', 'red', 'green', 'blue', 'cyan'])
    plt.xticks(y_pos, class_l, rotation=90)
    plt.ylabel('Images')
    plt.title('Images per Class')
    plt.show()

    display(disp_img_but)
    img_out = widgets.Output()
    display(img_out)
    def on_disp_button(b):
        with img_out:
            clear_output()
            display_images()
            #display_i()
    disp_img_but.on_click(on_disp_button)

In [75]:
def display_images():
    """Helper for displaying images from folder"""
    train_source = (ds_choice.source/'train/').ls().items
    for i, name in enumerate(train_source):
        fol = (ds_choice.source/name).ls().sorted()
        fol_disp = fol[0:5]
        filename = fol_disp.items
        fol_tensor = [tensor(Image.open(o)) for o in fol_disp]
        print(BOLD + BLUE + "Loc: " + RESET + str(name) + " " + BOLD + BLUE + "Number of Images: " + RESET +
              BOLD + RED + str(len(fol)))

        fig = plt.figure(figsize=(15,15))
        columns = 5
        rows = 1
        ax = []

        for i in range(columns*rows):
            for i, j in enumerate(fol_tensor):
                img = fol_tensor[i]    # create subplot and append to ax
                ax.append( fig.add_subplot(rows, columns, i+1))
                ax[-1].set_title("ax:"+str(filename[i]))  # set title
                plt.tick_params(bottom="on", left="on")
                plt.imshow(img)
                plt.xticks([])
        plt.show()


In [85]:
def display_ui():
    """ Display tabs for visual display"""
    button = widgets.Button(description="Train")
    button_b = widgets.Button(description="Metrics")
    button_m = widgets.Button(description='Model')
    button_l = widgets.Button(description='LR')

    test_button = widgets.Button(description='Batch')
    test2_button = widgets.Button(description='Test2')

    out1a = widgets.Output()
    out1 = widgets.Output()
    out2 = widgets.Output()
    out3 = widgets.Output()
    out4 = widgets.Output()
    out5 = widgets.Output()

    data1a = pd.DataFrame(np.random.normal(size = 50))
    data1 = pd.DataFrame(np.random.normal(size = 100))
    data2 = pd.DataFrame(np.random.normal(size = 150))
    data3 = pd.DataFrame(np.random.normal(size = 200))
    data4 = pd.DataFrame(np.random.normal(size = 250))
    data5 = pd.DataFrame(np.random.normal(size = 300))

    with out1a: #info
        clear_output()
        dashboard_one()

    with out1: #data
        clear_output()
        dashboard_two()

    with out2: #augmentation
        clear_output()
        #aug_dash()

    with out3: #Block
        clear_output()
        #ds_3()

    with out4: #code
        clear_output()
        #write_code()

    with out5: #Imagewoof Play
        clear_output()
        #print(BOLD + BLUE + 'Work in progress.....')
        #play_button = widgets.Button(description='Parameters')
        #display(play_button)
        #play_out = widgets.Output()
        #display(play_out)
        #def button_play(b):
        #    with play_out:
        #        clear_output()
        #        play_info()
        #play_button.on_click(button_play)

    display_ui.tab = widgets.Tab(children = [out1a, out1, out2, out3, out4, out5])
    display_ui.tab.set_title(0, 'Info')
    display_ui.tab.set_title(1, 'Data')
    display_ui.tab.set_title(2, 'Augmentation')
    display_ui.tab.set_title(3, 'DataBlock')
    display_ui.tab.set_title(4, 'Code')
    display_ui.tab.set_title(5, 'ImageWoof Play')
    display(display_ui.tab)


In [86]:
display_ui()